In [ ]:
include("init.jl")
using TensorOperations
using TensorDecompositions
using Statistics
using LinearAlgebra
import Plots
import StatsPlots
import LightGraphs

# Tensor decompositions sort of explained for Colin

This little notebook is for me to play with the tensor packages in Julia; to try to remember some stuff from my degree; and to help Colin understand what the point of it all is.

First things first - there is a bug in Julia where `zeroes` is misspelt.

In [ ]:
zeroes = zeros

# to-do:
# fork nncp to give it a quieter option (disabling progress bar); verbose=false does some
# also make nncp quick
# look into corcondia scaling with R
# check corcondia code is correct


## Generating our tensor

The idea behind a tensor decomposition or factorisation is that there's a lot of degeneracy around in the world these days.

Say, for example, you generate a tensor from three vectors. Then, by definition, that tensor can be described by three vectors rather than having to describe the whole tensor.

The point of this is that the three vectors are more interesting than the tensor - they will describe the patterns that have led to that tensor being created.

Mathematically:

$$ a_{ijk} = \Sigma_{r=1}^R u_{ir} v_{jr} w_{kr} $$

where $R$ is the number of factors asked for in the algorithm.

In this notebook, we'll generate a tensor $a$ from $u,v,w$, and use a tensor decomposition to attempt to retrieve those vectors.

In [ ]:
DIMS = [100,20,10,3]
u,v,w,x = [rand(dim) for dim in DIMS]
z = rand(DIMS...)
a = zeroes(DIMS...)
@show a |> size

# This is using Einstein's summation convention - the whole tensor a is generated by iterating through all possible pairs 
@tensor a[i,j,k,l] = u[i] * v[j] * w[k] * x[l]# + 0.001*z[i,j,k]
@show mean(a)

In [ ]:
# F = candecomp(a, 1, (randn(10,1),randn(20,1),randn(30,1));
#     # NB: need to have 2D arrays for the guess; 1D isn't good enough
#     compute_error=true,
#     method=:ALS,
#     maxiter=100_000,
# );
F = nncp(a, 1;
    compute_error=true,
    maxiter=10,
);
F.factors

In [ ]:
[all(f .> 0) for f in F.factors]

In [ ]:
# Conversely, tensor operations wants 1D arrays
U,V,W,X = [f |> Iterators.flatten |> collect for f in F.factors]

In [ ]:
@tensor A[i,j,k,l] := U[i] * V[j] * W[k] * X[l];

In [ ]:
@show mean(abs.(A-a))
@show mean(a)
@show mean(A)
inds = rand(1:minimum(DIMS))
@show A[inds...]
@show a[inds...]
@show maximum(abs.(A-a))

# Nice. Easy-mode.
# NB: u,v,w aren't the same as U,V,W; but a and A are very similar

In [ ]:
StatsPlots.plot([StatsPlots.boxplot(vec,legend=:none) for vec in (u,U,v,V,w,W)]..., layout=(1,6))

# This is quite interesting: it gets the shapes of the distributions right, but the normalisation wrong.

## So what does this mean for graphs?

First, let's make an adjacency matrix that varies with time.

We make an undirected graph with no self-loops as our base.

In [ ]:
N = 100 # ignored if using SBM
TIMESTEPS = 10
DENSITY = 0.05 # unused atm
LINK_HIATUS_RATE = 0.1
#base_adj = Int.(rand(N,N) .< DENSITY) |> Symmetric
#base_adj = base_adj .& mod.(one(base_adj).+1,2) # get rid of self-loops;

# Let's make an actually interesting graph
# g = LightGraphs.barabasi_albert(N,2)
#g = LightGraphs.erdos_renyi(N,DENSITY)
g = LightGraphs.watts_strogatz(N,4,0.01)

# g = LightGraphs.stochastic_block_model(
#     [
#         3   2/N   2/N; # c[a,b] = mean number of neighbours between nodes in block a and block b (only pay attention to upper tri)
#         0   3   2/N;
#         0   0   3;
#     ],
#     [50,50,50] # number of nodes in each block
# )

N = LightGraphs.nv(g)
base_adj = g |> LightGraphs.adjacency_matrix;

At each timestep, we temporarily turn off LINK_HIATUS_RATE links, and make a nice tensor.

In [ ]:
import ProgressMeter

In [ ]:
perms = Int.(rand(N,N,TIMESTEPS) .> LINK_HIATUS_RATE)
for t in 1:TIMESTEPS
    perms[:,:,t] = perms[:,:,t]  |> Symmetric
end

# Want to make a more recognisable temporal pattern
# Mask a block off and turn it on and off at various times
 perms = zeroes(Int,N,N,TIMESTEPS)
SPECIAL_NODES = N ÷ 10
SPECIAL_TIME = 3
#for i in 1:N, j in 1:N, t in 1:TIMESTEPS
#    perms[i,j,t] = i <= SPECIAL_NODES && j <= SPECIAL_NODES && t <= SPECIAL_TIME ? 0 : 1
#end
# should probably find a nicer way of doing this...

ADJ = zeroes(N,N,TIMESTEPS) # making this takes _AGES_
#for i in 1:N, j in 1:N, t in 1:TIMESTEPS
#    ADJ[i,j,t] = base_adj[i,j] & perms[i,j,t]
#end

for t in 1:TIMESTEPS
    ADJ[:,:,t] = base_adj
end

# I expected
# @tensor ADJ[i,j,t] = base_adj[i,j] & perms[i,j,t]
# to work, but it doesn't.

# should probably try to make more distinct temporal communities that agree with SBM to make this a fairer test

ADJ |> size

In [ ]:
using TensorToolbox
function quick_kron(ts,X)
    i_x = 1:(size(X) |> length) |> collect
    Y = similar(X)
    for (k,t) in reverse(ts |> enumerate |> collect)
        Y = ttm(X,t,k)
        X = permutedims(Y,reverse(i_x))
    end
    Y
end
    
function CORCONDIER(X,factors,R)
    us = []
    ss = []
    vs = []
#   @show factors
#   k = 0.70710678
#   factors = [ones(2,2), ones(2,2).*k, ones(2,2).*k]
#   @show factors
# python gives us different factors
    # that explains our discrepancy in corcondia
    for f in factors
        (u,s,v) = svd(f)
        push!(us,u)
        push!(ss,s)
        push!(vs,v)
    end

    Y = quick_kron(transpose.(us),X)
    # colin claims quick_kron is identical to slow
    # provided you reverse the order of the matrices
    Z = quick_kron((inv ∘ Diagonal).(ss),Y)
    #inds = [R for _ in 1:length(factors)]
    G = quick_kron(vs,Z)
#   @show get_G(X, factors)
    # all this is equivalent to G = reshape(reduce(kron, factors) |> pinv * X[:], inds...)
    
    # people often do this * 100 "to make it into percent"
    # but I find it easier to read as a decimal
    (1 - mapreduce(k -> (G[k] - δ(Tuple(k)...))^2,+,pairs(IndexCartesian(),G)|>keys)/R)
    #[G[i,j,k] - δ(i,j,k) for i in 1:R, j in 1:R, k in 1:R].^2 |> sum
    # how the dickens do we reduce this?
end

In [ ]:
r = 3
F = nncp(ADJ, r; # R is the number of components ≈ number of communities
    # I think Ciro spoke about optimising this number somewhere
    # papers refer to it - core consistency
    compute_error=true,
    maxiter=1000,
    verbose=false
);
#methods(nncp)
CORCONDIER(ADJ,F.factors,r)

# getting NaN on real data, fabulous

In [ ]:
using Serialization
adj_mats = deserialize("../data/processed/elm-adj-mats.jls")
X = Array{Float64}(undef, (size(adj_mats[1])..., length(adj_mats)))
[X[:,:,i]=f for (i,f) in adj_mats|>enumerate];

In [ ]:
function normalize_factors(factors)
    [factor ./ (map(norm, eachcol(factors[1])) |> permutedims) for factor in factors]
end

In [ ]:
# Compute a variety of Fits for X. Distribute it so that we can keep using the notebook while this is executing.
using Distributed
f = @spawn begin
    Fs = [F=nncp(X, r, verbose=false) for r in 1:1];
    Cs = [CORCONDIER(X,F.factors,r) for (r,F) in enumerate(Fs)];
    NCs = [CORCONDIER(X,F.factors|>normalize_factors,r) for (r,F) in enumerate(Fs)];
    Fs, Cs, NCs
end;

In [ ]:
# Run this when it's done
Fs,Cs,NCs = fetch(f) .|> x->x[1]

In [ ]:
# this cell takes a while (and makes the PC hang for a little bit)

R = TIMESTEPS # corcondia scales badly with R, so think before you run this
cc = []
for r in 1:R
    try
    @time F = nncp(ADJ, r; # R is the number of components ≈ number of communities
        # I think Ciro spoke about optimising this number somewhere
        # papers refer to it - core consistency
        compute_error=true,
        maxiter=1000,
    );
    push!(cc,(r,CORCONDIA(ADJ,F.factors,r)))
    catch
        ; # Ignore convergence issues etc
    end
end

In [ ]:
Plots.plot([c[1] for c in cc], [c[2] for c in cc]) # Hmm. Can't have more communities than timesteps.
# That's a bit weird.

# function is so bouncy that it looks pretty suspect 
# corcondia performs badly with large R too, it seems

# I'm beginning to suspect that it's just BS.

# Maybe we should look at silhouette instead?

# Hmm, since it's so bouncy, should we be run it a few times till we get the best score?
# Optimise our random seed as well as R?

In [ ]:
cc # Shouldn't R=1 always be perfect? I think something is wrong with our CORCONDIA

In [ ]:
R = 3
@time F = nncp(ADJ, R; # R is the number of components ≈ number of communities
    
    # I think Ciro spoke about optimising this number somewhere
    # papers refer to it - core consistency
    compute_error=true,
    maxiter=1000,
);
# (each u*v*w[:,r] is a component; each of u,v,w is a factor)
# n = 1000 -> 2 seconds
# n = 10,000 -> bloody ages. severely limited by i/o - using 25% CPU only. has been like, 5 minutes. Am bored.

In [ ]:
F.factors[3] |> size

In [ ]:
F.factors[1] # The largest numbers in each nodes correspond to the nodes most active in that community
(u,v,w) = [f for f in F.factors]

In [ ]:
Plots.plot(F.factors[3]) # You can see that some communities are inactive for the first few timesteps, as we'd expect

In [ ]:
Plots.heatmap(u)

In [ ]:
import GraphPlot
import Colors

In [ ]:
function show_communities(X, F, r)
    N = size(X)[1]
    u = F.factors[1]
    adj = X[:,:,size(X)[3]] |> BitArray
    
    if ((adj .| adj') .== adj) |> all
        g = LightGraphs.Graph(adj)
    else
        g = LightGraphs.DiGraph(adj)
    end
    
    colours = Colors.distinguishable_colors(r,Colors.colorant"blue")
    nodefillarr = []
    for n in 1:N
        ind = findmax(u[n,:])[2]
        push!(nodefillarr,colours[ind])
    end
    
    if N > 300
        proc = GraphPlot.gplothtml(g;nodefillc=nodefillarr)
        print("http://blanthornpc:2015/", split(proc.cmd.exec[2], "/")[3])
    else
        GraphPlot.gplot(g;nodefillc=nodefillarr)
    end
end

In [ ]:
show_communities(ADJ, F, 3)

In [ ]:
undir = X -> begin X = BitArray(X); (X .| X') |> Array{Float64} end
Xundir = similar(X)
[Xundir[:,:,i] = undir(X[:,:,i]) for i in 1:size(X)[3]];

In [ ]:
# The undirected graph seems to have better community detection, looking purely at the graphplot of the final graph.
# TODO:
# Try plotting factors[2] instead?
r = 5
Fundir = nncp(Xundir, 5)

In [ ]:
proc = show_communities(Xundir, Fundir, 5)
# Start caddy in /tmp first
print("http://blanthornpc:2015/", split(proc.cmd.exec[2], "/")[3])

In [ ]:
show_communities(X, Fs[5], 5)

In [ ]:
community_size = N ÷ R
colours = Colors.distinguishable_colors(R,Colors.colorant"blue")
nodefillarr = []
for n in 1:N
    ind = findmax(u[n,:])[2]
    push!(nodefillarr,colours[ind])
end
GraphPlot.gplot(g;nodefillc=nodefillarr) # Sure, these kind of look like communities.
# Eugh, if you put too many nodes in, GraphPlot gives up trying to colour them
# Would be nice to colour nodes by strength of their association too.

## Network interpretation

Say you have a temporal adjacency matrix which you wish to decompose into $R$ communities:

$$ a_{ijt} = \Sigma_{r=1}^R u_{ir} v_{jr} w_{tr} $$

NB: Each of $u,v,w$ is roughly (self) column-wise orthogonal - i.e, if an entry is large in one column of, e.g, u, then it will be smaller in another column of u.

Therefore, for any edge $a_{ijt}$ to appear from node $j$ to $i$ at time $t$, it must be true that, for some $r$, $u_{ir} v_{jr} w_{tr} \approx 1$.

Ignoring the temporal factor for a moment, we can say that high-scoring nodes in $\boldsymbol{u}_{r}$ are probably linked to by nodes in $\boldsymbol{v}_{r}$ - all at once, i.e, each $r$ fuzzily denotes a community of nodes who link to each other: so, $u_{ir}$ denotes the strength

$w_{tr}$ has a similar meaning: the highest scoring times for each $r$ means that the high-scoring nodes in $u$ and $v$ are "turned on" and have more outgoing or incoming links at those times.

In an undirected graph, we expect $u$ and $v$ to look similar.

## Fiddling

You don't need to pay attention to stuff below this line.

In [ ]:
b = u * v'
@tensor a[i,j,k] = b[i,j] * w[k] + 0.001*z[i,j,k]

fixme = ones(1:TIMESTEPS)
@tensor testADJ[i,j,t] := base_adj[i,j] * fixme[t] + perms[i,j,t]

# I really don't understand the difference between these expressions
# Or why fixme is needed

In [ ]:
@show findfirst(x->false,[1])

# Sparse playground

In [ ]:
using SparseArrays

DIMS = [10,20,30]
u,v,w = [round.(rand(dim)) for dim in DIMS] .|> hcat .|> SparseMatrixCSC # rounding should actually be done later
z = rand(DIMS...)
a = zeroes(DIMS...) # |> sparse # Error - sparse tensors aren't in Julia. Consider adopting https://github.com/JuliaTensors/SparseTensors.jl ?
# Or we could write a wrapper for taco - https://github.com/tensor-compiler/taco
@show a |> size

# This is using Einstein's summation convention - the whole tensor a is generated by iterating through all possible pairs 
#@tensor a[i,j,k] = u[i] * v[j] * w[k]# + 0.001*z[i,j,k]
for i in 1:DIMS[1], j in 1:DIMS[2], k in 1:DIMS[3]
    a[i,j,k] = u[i] * v[j] * w[k]
end
@show mean(a)

In [ ]:
base_adj |> Array

In [ ]:
times = []
Ns = 10 .^(1:0.1:4) .|> round .|> Int
R = 10
for n in Ns
    ADJ = zeroes(n,n,1) # making this takes _AGES_
    g = LightGraphs.watts_strogatz(n,4,0.01)
    base_adj = g |> LightGraphs.adjacency_matrix;
    ADJ[:,:,1] = base_adj

    t = @elapsed F = nncp(ADJ, 1; # R is the number of components ≈ number of communities
        # I think Ciro spoke about optimising this number somewhere
        # papers refer to it - core consistency
        compute_error=true,
        maxiter=100,
    );
    push!(times,t)
end

In [ ]:
Plots.plotly()
Plots.scatter(Ns,times;
    xlabel = "N",
    ylabel = "Time taken / seconds",
    legend = :none,
    yscale = :log10,
    xscale = :log10
)

# tl;dr: power-law, N^k, where k is quite big

## Choosing R

https://github.com/alessandrobessi/corcondia/blob/master/coreconsistency.py <- CORCONDIA

In [ ]:
CORCONDIA(ADJ,F.factors,10)